# 영업 성공 여부 분류 경진대회

# 할 것

결측치 처리
파생변수 생성

## 1. 데이터 확인

### 필수 라이브러리

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def seed_everything(seed):
    np.random.seed(seed) #numpy를 사용할 경우 고정
seed_everything(42)

### 데이터 셋 읽어오기

In [2]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [3]:
df_train[:10]

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,idit_strategic_ver,customer_job,lead_desc_length,inquiry_type,product_category,product_subcategory,product_modelname,customer_country.1,customer_position,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.00,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,NaN,purchasing,62,Quotation or purchase consultation,multi-split,NaN,NaN,/Quezon City/Philippines,entry level,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.00,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,NaN,media and communication,96,Quotation or purchase consultation,multi-split,NaN,NaN,/PH-00/Philippines,ceo/founder,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.00,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,NaN,engineering,56,Product Information,single-split,NaN,NaN,/Kolkata /India,partner,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.00,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,NaN,entrepreneurship,44,Quotation or purchase consultation,vrf,NaN,NaN,/Bhubaneswar/India,ceo/founder,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.00,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,NaN,consulting,97,Quotation or purchase consultation,multi-split,NaN,NaN,/Hyderabad/India,partner,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True
5,1.00,/Abuja/Nigeria,AS,0.040816,16328,End-Customer,SMB,NaN,NaN,NaN,NaN,program and project management,1114,Quotation or purchase consultation,chiller,NaN,NaN,/Abuja/Nigeria,manager,LGEAF,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,5,True
6,0.75,"/Jeddah, KSA/Saudi Arabia",AS,0.040816,20664,End-Customer,SMB,NaN,NaN,NaN,NaN,engineering,420,Quotation or purchase consultation,single-split,NaN,NaN,"/Jeddah, KSA/Saudi Arabia",manager,LGESJ,NaN,1,0,0.003079,0.026846,corporate / office,Engineering,6,True
7,1.00,/Guwahati/India,AS,0.088889,17983,End-Customer,SMB,NaN,NaN,NaN,NaN,sales,205,Quotation or purchase consultation,vrf,NaN,NaN,/Guwahati/India,partner,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Manufacturing,7,True
8,0.75,/Cebu city/Philippines,AS,0.066667,30867,Specifier/ Influencer,Enterprise,3.0,NaN,NaN,NaN,other,103,Quotation or purchase consultation,multi-split,NaN,NaN,/Cebu city/Philippines,vice president,LGEPH,less than 3 months,0,0,0.003079,0.026846,corporate / office,Construction,8,True
9,0.75,"/hauz khas,delhi/India",AS,0.088889,6084,End-Customer,SMB,NaN,NaN,NaN,NaN,other,252,Quotation or purchase consultation,vrf,NaN,NaN,"/hauz khas,delhi/India",manager,LGEIL,3 months ~ 6 months,1,0,0.003079,0.026846,corporate / office,NaN,9,True


In [4]:
'response_corporate', 'product_category', 'customer_job', 'expected_timeline', 'enterprise', 'inquiry_type', 'product_modelname', 'business_unit', 'customer_position', 'customer_type', 'customer_country', 'product_subcategory'

('response_corporate',
 'product_category',
 'customer_job',
 'expected_timeline',
 'enterprise',
 'inquiry_type',
 'product_modelname',
 'business_unit',
 'customer_position',
 'customer_type',
 'customer_country',
 'product_subcategory')

In [5]:
df_train['business_unit'].unique()

array(['AS', 'ID', 'IT', 'Solution', 'CM'], dtype=object)

In [6]:
# 결측치가 있는 변수 및 결측률 파악 

mis_val = df_train.isnull().sum()
mis_val_bool = mis_val >= 1
mis_val_df = pd.concat([mis_val, mis_val_bool], axis = 1)
mis_val_df = mis_val_df.rename(columns = {0 : 'mis_val', 1 : 'mis_val_bool'})

mis_val_data = mis_val_df.loc[mis_val_df['mis_val_bool'] == True, :]
mis_val_data['ratio'] = mis_val_data['mis_val'] / 59299
mis_val_data_sorted = mis_val_data.sort_values(by='ratio', ascending=True)
mis_val_data_sorted

/tmp/ipykernel_335/929980573.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mis_val_data['ratio'] = mis_val_data['mis_val'] / 59299


,mis_val,mis_val_bool,ratio
inquiry_type,941,True,0.015869
customer_country.1,982,True,0.016560
customer_country,982,True,0.016560
customer_job,18733,True,0.315908
product_category,19374,True,0.326717
expected_timeline,30863,True,0.520464
business_area,40882,True,0.689421
ver_win_rate_x,40882,True,0.689421
customer_type,43961,True,0.741345
ver_win_ratio_per_bu,43995,True,0.741918


In [7]:
# idit_strategic_ver , it_strategic_ver , id_strategic_ver 세 columns 값들이 array([nan,  1.])
# historical_existing_cnt nan = 0

#inquiry_type - 대소문자 통합 -> 전부 소문자로

# print(df_train['customer_type'].str.lower().unique())
# print(df_train['customer_type'].unique())

In [8]:
# customer_country , customer_country.1의 값이 일치함.
df_train = df_train.drop(labels = ['customer_country.1'],axis=1)
df_test = df_test.drop(labels = ['customer_country.1','id'],axis=1)

dtypes = df_train.dtypes

# 각 열의 고유값 개수 확인
unique_value_counts = df_train.nunique()

# 수치형 변수가 아닌 컬럼을 저장할 리스트 초기화
object_columns = []

# 각 열에 대해 데이터 타입이 'object'이거나 고유값 개수가 일정 이하인 경우를 확인하여 non_numeric_columns 리스트에 추가
for col in df_train.columns:
    if dtypes[col] == 'object':
        object_columns.append(col)

# 수치형 변수가 아닌 컬럼 출력
print("범주형 컬럼:", object_columns)

범주형 컬럼: ['customer_country', 'business_unit', 'customer_type', 'enterprise', 'customer_job', 'inquiry_type', 'product_category', 'product_subcategory', 'product_modelname', 'customer_position', 'response_corporate', 'expected_timeline', 'business_area', 'business_subarea']


### 전처리
inquiry_type,customer_country - 대소문자 통합
customer_country 결측치 행 삭제

In [9]:
def preprocess_1(col):
    df_train[col] = df_train[col].str.lower()
    df_test[col] = df_test[col].str.lower()
    
    mask_train = df_train[col].notnull()
    mask_test = df_test[col].notnull()
    df_train.loc[mask_train, col] = df_train.loc[mask_train, col].apply(lambda x: x.replace(" ", ""))
    df_test.loc[mask_test, col] = df_test.loc[mask_test, col].apply(lambda x: x.replace(" ", ""))    

In [10]:
for n in object_columns:
    preprocess_1(n)
#test data에 customer_country = nan 이 없어서 행삭제 해도 될듯. 삭제양도 많지 않음.
df_train = df_train.dropna(subset=['customer_country'])

# 보류
# df_train['customer_country'] = df_train['customer_country'].apply(lambda x: x.split('/')[-1])
# df_test['customer_country'] = df_test['customer_country'].apply(lambda x: x.split('/')[-1])

In [11]:
print(df_train['inquiry_type'].unique())

['quotationorpurchaseconsultation' 'productinformation' 'other'
 'usageortechnicalconsultation' 'trainings' 'services' 'salesinquiry'
 'etc.' 'technicalsupport' 'technicalconsultation' 'requestforpartnership'
 nan 'sales' 'technical' 'other_' 'quotation_or_purchase_consultation'
 'requestademo' 'requestfordistributorship'
 'requestforquotationorpurchase' 'requestfortechnicalconsulting'
 '(selectid_needs)' 'aio' 'needs' 'purchase' 'technical_consultation'
 'customersuggestions' 'eventinquiry' 'others' 'oem/odmrequest'
 'hospitaltv' 'iwanttoknowthedetailsaboutit' 'educationalequipments'
 'digitalplatform' 'tvinteractive' 'teach' 'displaytextbookandphotos'
 'quotation_' 'purchaseorquotation' 'displayproduct' 'firstinfoandpricing'
 'estoybuscandoparaecuadoresteproductolgmagnitmicroled,paraunclientede138pulgadas,conenviómarítimo.'
 'holamepuedencotizar19pantallasinteractivasde100pulgadasentregadasenguayaquil-ecuador.'
 'probeamprecio' 'hoteltvproducts' 'pantallasinteractivasparaclinicas'
 '

### 레이블 인코딩

In [ ]:
# def label_encoding(series: pd.Series) -> pd.Series:
#     """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

#     my_dict = {}

#     # 모든 요소를 문자열로 변환
#     series = series.astype(str)

#     for idx, value in enumerate(sorted(series.unique())):
#         my_dict[value] = idx
#     series = series.map(my_dict)

#     return series

In [ ]:
# # 레이블 인코딩할 칼럼들
# label_columns = [
#     "customer_country",
#     "business_subarea",
#     "business_area",
#     "business_unit",
#     "customer_type",
#     "enterprise",
#     "customer_job",
#     "inquiry_type",
#     "product_category",
#     "product_subcategory",
#     "product_modelname",
#     "customer_position",
#     "response_corporate",
#     "expected_timeline",
# ]

# df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

# for col in label_columns:
#     df_all[col] = label_encoding(df_all[col])

In [ ]:
# df_all.head()

In [ ]:
# for col in label_columns:  
#     df_train[col] = df_all.iloc[: len(df_train)][col]
#     df_test[col] = df_all.iloc[len(df_train):][col]

## 특징추출

In [ ]:
# import catboost
# from catboost import CatBoostClassifier
# from sklearn.model_selection import StratifiedKFold

# stf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)

# result = []

# #y_label을 제외한 컬럼
# x_columns = list(set(df_train.columns)-set('is_converted'))

# train_x = df_train.drop(columns=['is_converted'])
# train_y = df_train['is_converted']

# n = 0

# for train_index, valid_index in stf.split(train_x, train_y):
#     train_X = np.array(train_x)
#     train_Y = np.array(train_y)

#     X_train, X_valid = train_X[train_index], train_X[valid_index]
#     y_train, y_valid = train_Y[train_index], train_Y[valid_index]

#     model = CatBoostClassifier(iterations = 1000, random_seed=42,
#                                verbose=0)

#     summary = model.select_features(
#         X = X_train,
#         y = y_train,
#         features_for_select = '0-22',
#         eval_set=(X_valid, y_valid),
#         num_features_to_select=20,
#         steps=13,
#         train_final_model=False,
#         logging_level='Silent',
#         plot=False
#     )

#     result.append(summary['selected_features'])
#     print(n)
#     n+=1
# print("done")

In [12]:
# feature_selected = []
# for i in range(5):
#     for j in result[i]:
#         feature_selected.append(j)
feature_selected = ['bant_submit', 'customer_country', 'business_unit', 'com_reg_ver_win_rate', 'customer_idx', 'customer_type', 'enterprise', 'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver', 'customer_job', 'lead_desc_length', 'inquiry_type', 'product_category', 'product_subcategory', 'product_modelname', 'customer_position', 'response_corporate', 'expected_timeline', 'ver_cus', 'ver_pro', 'ver_win_rate_x','is_converted']

# feature_selected에 있는 각 열 이름에 대해 열 인덱스를 구합니다.
feature_selected_index = [df_train.columns.get_loc(col) for col in feature_selected]

# 각 열 인덱스를 사용하여 열을 선택합니다.
feature_selected_columns = df_train.columns[feature_selected_index]

# 선택된 열을 출력합니다.
print('선택된 columns : ', len(feature_selected_columns), '\n', feature_selected_columns, '\n\n')

# 선택되지 않은 열을 구합니다.
feature_nonSelected_columns = list(set(df_train.columns) - set(feature_selected_columns))

# 선택되지 않은 열을 출력합니다.
print('선택되지 않은 columns : ', len(feature_nonSelected_columns), '\n', feature_nonSelected_columns)

# feature_selected = [df_train.columns[i] for i in feature_selected]
# feature_nonSelected = list(set(train_x.columns) - set(feature_selected))
# print('선택된 columns : ',len(feature_selected),'\n',feature_selected,'\n\n')
# print('선택된 columns : ',len(feature_nonSelected),'\n',feature_nonSelected)


선택된 columns :  24 
 Index(['bant_submit', 'customer_country', 'business_unit',
       'com_reg_ver_win_rate', 'customer_idx', 'customer_type', 'enterprise',
       'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver',
       'idit_strategic_ver', 'customer_job', 'lead_desc_length',
       'inquiry_type', 'product_category', 'product_subcategory',
       'product_modelname', 'customer_position', 'response_corporate',
       'expected_timeline', 'ver_cus', 'ver_pro', 'ver_win_rate_x',
       'is_converted'],
      dtype='object') 


선택되지 않은 columns :  4 
 ['ver_win_ratio_per_bu', 'business_subarea', 'lead_owner', 'business_area']


In [13]:
df_train = df_train[feature_selected]
df_test = df_test[feature_selected]

t = df_train.copy()
corr = t.corr()
corr.style.background_gradient(cmap = 'coolwarm')

ValueError: could not convert string to float: '/quezoncity/philippines'

In [16]:
print(len(df_train.columns),'\n',df_train.columns)
print('\n\n\n')
print(len(df_test.columns),'\n',df_test.columns)

24 
 Index(['bant_submit', 'customer_country', 'business_unit',
       'com_reg_ver_win_rate', 'customer_idx', 'customer_type', 'enterprise',
       'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver',
       'idit_strategic_ver', 'customer_job', 'lead_desc_length',
       'inquiry_type', 'product_category', 'product_subcategory',
       'product_modelname', 'customer_position', 'response_corporate',
       'expected_timeline', 'ver_cus', 'ver_pro', 'ver_win_rate_x',
       'is_converted'],
      dtype='object')




24 
 Index(['bant_submit', 'customer_country', 'business_unit',
       'com_reg_ver_win_rate', 'customer_idx', 'customer_type', 'enterprise',
       'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver',
       'idit_strategic_ver', 'customer_job', 'lead_desc_length',
       'inquiry_type', 'product_category', 'product_subcategory',
       'product_modelname', 'customer_position', 'response_corporate',
       'expected_timeline', 'ver_cus', 'ver_pr

## 3. 모델 학습

In [25]:
y = pd.DataFrame([[ True],
 [ True],
 [ True],
 [False],
 [False],
 [False]])

print(y)

       0
0   True
1   True
2   True
3  False
4  False
5  False


### 모델 정의 

In [42]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

# Stratified K-Fold 교차 검증 설정
n_splits = 5  # 원하는 Fold 수
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

cat_features = [1,2, 5, 6,11,13,14,15,16,17,18,19]

# 최적의 하이퍼파라미터 설정
best_params = {'depth': 10, 'iterations': 1500, 'learning_rate': 0.01}

# 교차 검증 수행 및 평가 지표 저장
macro_f1_scores = []
probabilities_list = []
best_f1_score = 0

train_x = df_train.drop(columns=['is_converted'])
train_y = df_train['is_converted']
train_y = pd.DataFrame(train_y,columns=['is_converted'])

train_x = train_x.fillna(0)

X = train_x.values  # 특성 데이터
y = train_y.values  # 라벨 데이터

for train_index, valid_index in skf.split(X, y):
    X_train, X_valid = X[train_index], X[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]

    # CatBoost 모델 생성 및 학습
    model = CatBoostClassifier(loss_function='Logloss', cat_features=cat_features, eval_metric='F1', 
                               random_seed=42, verbose=500, **best_params)
    
    model.fit(X_train, y_train)

    # 검증 데이터에 대한 예측 및 평가 (Macro F1 Score)
    valid_predictions = model.predict(X_valid)
    valid_predictions = np.array([bool_mapping[item] for item in valid_predictions])
    valid_predictions = arr_bool.reshape(-1, 1)
    
#     print(valid_predictions,'\n\n\n')
#     print(y_valid)
    
    macro_f1 = f1_score(y_valid, valid_predictions, average='macro')
    macro_f1_scores.append(macro_f1)

    if macro_f1 > best_f1_score:
        best_f1_score = macro_f1
        best_classification_model = model

# Macro F1 Score 출력
print("Macro F1 Scores:", macro_f1_scores)


0:	learn: 0.6422722	total: 127ms	remaining: 3m 10s
500:	learn: 0.7673853	total: 1m 4s	remaining: 2m 9s
1000:	learn: 0.8215999	total: 2m 12s	remaining: 1m 5s
1499:	learn: 0.8653159	total: 3m 22s	remaining: 0us
0:	learn: 0.6702949	total: 169ms	remaining: 4m 13s
500:	learn: 0.7683818	total: 1m 4s	remaining: 2m 9s
1000:	learn: 0.8265353	total: 2m 16s	remaining: 1m 7s
1499:	learn: 0.8614512	total: 3m 27s	remaining: 0us
0:	learn: 0.6671378	total: 143ms	remaining: 3m 34s
500:	learn: 0.7624633	total: 1m 5s	remaining: 2m 10s
1000:	learn: 0.8193108	total: 2m 18s	remaining: 1m 8s
1499:	learn: 0.8618421	total: 3m 29s	remaining: 0us


ValueError: Found input variables with inconsistent numbers of samples: [11663, 11664]

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

# Stratified K-Fold 교차 검증 설정
n_splits = 5  # 원하는 Fold 수
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

cat_features = [1,2, 5, 6,11,13,14,15,16,17,18,19]

# 최적의 하이퍼파라미터 설정
best_params = {'depth': 10, 'iterations': 1500, 'learning_rate': 0.01}

# 교차 검증 수행 및 평가 지표 저장
macro_f1_scores = []
probabilities_list = []
best_f1_score = 0

train_x = df_train.drop(columns=['is_converted'])
train_y = df_train['is_converted']
train_y = pd.DataFrame(train_y,columns=['is_converted'])

train_x = train_x.fillna(0)

X = train_x.values  # 특성 데이터
y = train_y.values  # 라벨 데이터

train_index, valid_index = skf.split(X, y)
X_train, X_valid = X[train_index], X[valid_index]
y_train, y_valid = y[train_index], y[valid_index]

# CatBoost 모델 생성 및 학습
model = CatBoostClassifier(loss_function='Logloss', cat_features=cat_features, eval_metric='F1', 
                           random_seed=42, verbose=500, **best_params)

model.fit(X_train, y_train)

# 검증 데이터에 대한 예측 및 평가 (Macro F1 Score)
valid_predictions = model.predict(X_valid)
print(valid_predictions,'\n\n\n')
print(y_valid)

macro_f1 = f1_score(y_valid, valid_predictions, average='macro')
macro_f1_scores.append(macro_f1)

if macro_f1 > best_f1_score:
    best_f1_score = macro_f1
    best_classification_model = model

# Macro F1 Score 출력
print("Macro F1 Scores:", macro_f1_scores)


In [36]:
bool_mapping = {'True': True, 'False': False}

# 매핑된 값을 사용하여 배열을 불리언 유형으로 변환
arr_bool = np.array([bool_mapping[item] for item in valid_predictions])

print(arr_bool)

[False False False ... False False False]


In [41]:
arr_bool.reshape(-1, 1)


array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [58]:
print(len(train_x))
print(len(train_y))

58317
58317


### 모델 학습

In [43]:
best_params = {'depth': 10, 'iterations': 1500, 'learning_rate': 0.01}

clf = CatBoostClassifier(loss_function='Logloss', cat_features=cat_features, eval_metric='F1', 
                           random_seed=42, verbose=500, **best_params)

In [48]:
clf.fit(X, y)

0:	learn: 0.6689993	total: 244ms	remaining: 6m 6s
500:	learn: 0.7779204	total: 1m 19s	remaining: 2m 38s
1000:	learn: 0.8237183	total: 2m 45s	remaining: 1m 22s
1499:	learn: 0.8604289	total: 4m 10s	remaining: 0us


### 모델 성능 보기

In [61]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [60]:
pred = model.predict(x_val.fillna(0))
print(y_val.info(),'\n')
print(type(pred))

NameError: name 'x_val' is not defined

In [ ]:
# pred를 판다스 Series로 변환
pred_series = pd.Series(pred, index=y_val.index)
pred_series = pred_series.astype(bool)
y_val = y_val.astype(bool)
print(pred_series.head())
print(y_val.head())

pred = model.predict(x_val.fillna(0))
get_clf_eval(y_val, pred)

## 4. 제출하기

### 테스트 데이터 예측

In [62]:
# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted"], axis=1)

In [65]:
test_pred = model.predict(x_test.fillna(0))
print(test_pred)
sum(int(pred == 'True') for pred in test_pred)

['False' 'False' 'False' ... 'False' 'False' 'False']


277

### 제출 파일 작성

In [ ]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**